In [1]:
import numpy as np
import os
import pandas as pd
from tqdm import tqdm_notebook as tqdm
from xml.dom import minidom

In [2]:
img_dir='../../../../Data/VOCdevkit/VOC2012/JPEGImages/'
class_label_dir='../../../../Data/VOCdevkit/VOC2012/ImageSets/Main/'
anotation_dir='../../../../Data/VOCdevkit/VOC2012/Annotations/'
seg_class_dir='../../../../Data/VOCdevkit/VOC2012/SegmentationClass/'
seg_obj_dir='../../../../Data/VOCdevkit/VOC2012/SegmentationObject/'

## 找到图像对应的label 

In [71]:
categories=[]
for root,dirs,files in os.walk(class_label_dir):
    for file in files:
        if 'trainval' in file and '_' in file:
            categories.append(file.split('_')[0])

In [72]:
categories

['dog',
 'sofa',
 'bicycle',
 'aeroplane',
 'pottedplant',
 'car',
 'horse',
 'bird',
 'bottle',
 'cow',
 'cat',
 'tvmonitor',
 'person',
 'train',
 'boat',
 'diningtable',
 'sheep',
 'chair',
 'motorbike',
 'bus']

In [73]:
img_with_label={}
for i in range(len(categories)):
    cate=class_label_dir+categories[i]+'_trainval.txt'
    with open(cate,'r') as f:
        data=f.readlines()
        
        for line in data:
            items=line.split(' ')
            if items[0] in img_with_label.keys():
                if int(items[-1])==-1:
                    img_with_label[items[0]][i]=0
                else:
                    img_with_label[items[0]][i]=1
            else:
                img_with_label[items[0]]=[-1 for _ in range(20)]
                if int(items[-1])==-1:
                    img_with_label[items[0]][i]=0
                else:
                    img_with_label[items[0]][i]=1

# Create Dataframe for all imgs

In [74]:
imgs=[]
for root,dirs,files in os.walk(img_dir):
    for file in files:
        imgs.append(file.split('.')[0])

In [75]:
img_classes=[]
for img in imgs:
    if img in img_with_label.keys():
        img_classes.append(img_with_label[img])
    else:
        img_classes.append(np.nan)

In [76]:
img_bbox=[]
img_bbox_label=[]
for img in tqdm(imgs):
    if os.path.exists(anotation_dir+img+'.xml'):
        single_img_bbox=[]
        single_bbox_label=[]
        
        mydoc = minidom.parse(anotation_dir+img+'.xml')
        root=mydoc.documentElement
        
        items = root.getElementsByTagName('object')
        for item in items:
            single_bbox=[]
            name=item.getElementsByTagName('name')[0].firstChild.data
            bbox=item.getElementsByTagName('bndbox')[0]

            for b in bbox.childNodes:
                if b.nodeType==1:
                    single_bbox.append(b.firstChild.data)
                    
            single_img_bbox.append(single_bbox)
            single_bbox_label.append(name)
        
        img_bbox.append(single_img_bbox)
        img_bbox_label.append(single_bbox_label)
    else:
        img_bbox.append(np.nan)
        img_bbox_label.append(np.nan)

In [77]:
img_seg_class=[]
for img in tqdm(imgs):
    if os.path.exists(seg_class_dir+img+'.png'):
        img_seg_class.append(1)
    else:
        img_seg_class.append(np.nan)

In [78]:
img_seg_obj=[]
for img in tqdm(imgs):
    if os.path.exists(seg_obj_dir+img+'.png'):
        img_seg_obj.append(1)
    else:
        img_seg_obj.append(np.nan)

In [79]:
data_csv=pd.DataFrame({'img_name':imgs})

In [80]:
data_csv['img_class']=img_classes

In [81]:
data_csv['img_bbox']=img_bbox

In [82]:
data_csv['seg_class']=img_seg_class

In [83]:
data_csv['seg_obj']=img_seg_obj

In [84]:
data_csv['bbox_label']=img_bbox_label

In [85]:
data_csv.head()

,img_name,img_class,img_bbox,seg_class,seg_obj,bbox_label
0,2010_004157,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, ...","[[499, 1, 293, 19]]",NaN,NaN,[cat]
1,2012_003811,NaN,"[[56, 11, 277, 151]]",NaN,NaN,[person]
2,2007_007048,NaN,"[[227, 55, 500, 324], [82, 56, 245, 257], [76,...",1.0,1.0,"[sheep, sheep, bottle, person, person, person,..."
3,2012_001460,NaN,"[[230, 142, 229, 53]]",NaN,NaN,[person]
4,2008_006392,"[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, ...","[[1, 228, 95, 343], [77, 198, 101, 266], [127,...",NaN,NaN,"[car, person, person, person, person]"


## delete len(img_bbox)!= img_class 

In [86]:
def return_one(arr):
    res=0
    for i in arr:
        if i==1:
            res+=1
    
    return res

In [88]:
del_index=[]
for index in data_csv.index:
    if type(data_csv.at[index,'img_class'])==type([1,2,3]) and type(data_csv.at[index,'bbox_label'])==type(['a']):
        if return_one(data_csv.at[index,'img_class'])!=len(set(data_csv.at[index,'bbox_label'])):
            del_index.append(index)

In [90]:
len(del_index)

0

In [104]:
data_csv.to_csv('data.csv',index=False)

In [102]:
sea_index=[]
for index in data_csv.index:
    if type(data_csv.at[index,'img_class'])==type([1,2,3]):
        if data_csv.at[index,'img_class'][12]==1:
            sea_index.append(index)

In [103]:
data_csv.loc[sea_index]

,img_name,img_class,img_bbox,seg_class,seg_obj,bbox_label
4,2008_006392,"[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, ...","[[1, 228, 95, 343], [77, 198, 101, 266], [127,...",NaN,NaN,"[car, person, person, person, person]"
6,2008_006014,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, ...","[[199, 175, 283, 194], [251, 169, 272, 188], [...",NaN,NaN,"[boat, person, person, person, person, person]"
8,2011_000888,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ...","[[435, 11, 290, 64], [369, 332, 177, 144]]",1.0,1.0,"[bus, person]"
10,2008_003447,"[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, ...","[[118, 82, 393, 366], [204, 129, 283, 374]]",NaN,NaN,"[horse, person]"
11,2008_002115,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ...","[[21, 21, 262, 340], [231, 1, 500, 319]]",NaN,NaN,"[person, motorbike]"
15,2009_004677,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, ...","[[472, 1, 336, 92], [500, 488, 210, 155], [487...",NaN,NaN,"[train, person, person]"
25,2008_002787,"[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ...","[[35, 170, 231, 375], [58, 26, 206, 325], [358...",NaN,NaN,"[bicycle, person, person, bicycle]"
32,2008_001083,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ...","[[204, 107, 267, 168], [21, 138, 204, 333], [8...",NaN,NaN,"[person, person, person, person, person, chair..."
33,2008_006611,"[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ...","[[419, 190, 500, 343], [1, 183, 157, 343], [16...",NaN,NaN,"[chair, chair, chair, chair, diningtable, pers..."
35,2009_003266,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ...","[[491, 313, 332, 31], [317, 152, 332, 55], [18...",NaN,NaN,"[person, person, person, person, person, perso..."


In [105]:
improved_img_class=[]
for index in data_csv.index:
    if type(data_csv.loc[index,'img_class'])!=type([1,2,3]):
        if type(data_csv.loc[index,'bbox_label'])==type(['a']):
            cate_arr=[]
            
            classes=set(data_csv.loc[index,'bbox_label'])
            for cate in categories:
                if cate in classes:
                    cate_arr.append(1)
                else:
                    cate_arr.append(0)
        improved_img_class.append(cate_arr)
    
    else:
        improved_img_class.append(data_csv.loc[index,'img_class'])

In [106]:
len(improved_img_class)

17125

In [107]:
data_csv['img_class']=improved_img_class

In [108]:
data_csv.head()

,img_name,img_class,img_bbox,seg_class,seg_obj,bbox_label
0,2010_004157,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, ...","[[499, 1, 293, 19]]",NaN,NaN,[cat]
1,2012_003811,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ...","[[56, 11, 277, 151]]",NaN,NaN,[person]
2,2007_007048,"[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, ...","[[227, 55, 500, 324], [82, 56, 245, 257], [76,...",1.0,1.0,"[sheep, sheep, bottle, person, person, person,..."
3,2012_001460,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ...","[[230, 142, 229, 53]]",NaN,NaN,[person]
4,2008_006392,"[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, ...","[[1, 228, 95, 343], [77, 198, 101, 266], [127,...",NaN,NaN,"[car, person, person, person, person]"


In [109]:
data_csv.to_csv('improved_data.csv',index=False)

In [111]:
len(data_csv[data_csv['seg_class'].isnull() & data_csv['img_class'].isnull()])

0

# 切分图像

In [3]:
data_csv=pd.read_csv('improved_data.csv')

In [4]:
data_csv.head()

,img_name,img_class,img_bbox,seg_class,seg_obj,bbox_label
0,2010_004157,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, ...","[['499', '1', '293', '19']]",NaN,NaN,['cat']
1,2012_003811,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ...","[['56', '11', '277', '151']]",NaN,NaN,['person']
2,2007_007048,"[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, ...","[['227', '55', '500', '324'], ['82', '56', '24...",1.0,1.0,"['sheep', 'sheep', 'bottle', 'person', 'person..."
3,2012_001460,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ...","[['230', '142', '229', '53']]",NaN,NaN,['person']
4,2008_006392,"[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, ...","[['1', '228', '95', '343'], ['77', '198', '101...",NaN,NaN,"['car', 'person', 'person', 'person', 'person']"


In [5]:
class_seg_data=data_csv[data_csv['seg_class'].notnull()]

In [6]:
obj_seg_data=data_csv[data_csv['seg_obj'].notnull()]

In [11]:
(obj_seg_data.index==class_seg_data.index).all()==True

True

In [12]:
class_only_data=data_csv.drop(class_seg_data.index)

In [13]:
class_only_data.head()

,img_name,img_class,img_bbox,seg_class,seg_obj,bbox_label
0,2010_004157,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, ...","[['499', '1', '293', '19']]",NaN,NaN,['cat']
1,2012_003811,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ...","[['56', '11', '277', '151']]",NaN,NaN,['person']
3,2012_001460,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ...","[['230', '142', '229', '53']]",NaN,NaN,['person']
4,2008_006392,"[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, ...","[['1', '228', '95', '343'], ['77', '198', '101...",NaN,NaN,"['car', 'person', 'person', 'person', 'person']"
5,2010_001979,"[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[['395', '123', '265', '88']]",NaN,NaN,['aeroplane']


In [14]:
class_seg_data.to_csv('class_seg_data.csv',index=False)
obj_seg_data.to_csv('obj_seg_data.csv',index=False)
class_only_data.to_csv('class_only_data.csv',index=False)